In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os
import time
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.utils.data import TensorDataset, DataLoader
from sklearn.preprocessing import MinMaxScaler  
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report

In [ ]:
# Store csv file in a Pandas DataFrame
df = pd.read_csv('combined_labled_k6.csv')
df.head()

In [ ]:
y = pd.get_dummies(df.cluster, prefix='base')
y.rename(columns = {'base_0':'base_line', 'base_1':'rotation', 'base_2':'translation'}, inplace = True)
print(y.head())

In [ ]:
sns.countplot( x='rotation', data=y)

In [ ]:
y= np.array(y)
print(y)

In [ ]:
X = df.iloc[:, 1:-1]

X=np.array(X)
print(X)

In [ ]:

y

In [ ]:
X_tr, X_te, y_tr, y_te = train_test_split(X, y, test_size=0.2, random_state=None,shuffle=False)

In [ ]:
scaler = MinMaxScaler()
X1= scaler.fit_transform(X)
x1 = scaler.transform(X1)

print(X1.shape)




In [ ]:
# sc = MinMaxScaler()

# y1= sc.fit_transform(y.reshape(-1,1))
# #y1 = sc.transform(y1)


print(y.shape)
print(y[3])

In [ ]:
print(X1)

In [ ]:
## train dataclass TrainData(Dataset):
#from http.client import _DataType


class TrainData(Dataset):   
    def __init__(self, X_data, y_data):
        self.X_data = X_data
        self.y_data = y_data
        self.window=5
        
    def __getitem__(self, index):
        if index>= self.length-self.window:
            X=self.X_data[index:index+self.window,]
            Y=self.y_data[index,]
        else:
            X=self.X_data[index:index+self.window,]  
            Y=self.y_data[index+self.window,]
        
        return X,Y
        
    def __len__ (self):
        Q = len(self.X_data)
        self.length=Q-self.window+1
        return self.length

# train_data = TrainData(torch.FloatTensor(X_train), 
#                        torch.FloatTensor(y_train))

## test data  
# class TestData(Dataset):
    
#     def __init__(self, X_data,y_data):
#         self.X_data = X_data
#         self.y_data = y_data
#         self.window=5
#     def __getitem__(self, index):
#         if index>= self.length-self.window:
#             X=self.X_data[index:index+self.window]
#             Y=self.y_data[index,]

#         else:
#             X=self.X_data[index:index+self.window]  
#             Y=self.y_data[index+self.window,]
        
#         return X,Y
#     def __len__ (self):
#         Q = len(self.X_data)
#         self.length=Q-self.window+1
#         return self.length
    

#test_data = TrainData(torch.FloatTensor(X_test),torch.FloatTensor(y_test))                      

In [ ]:
cs=TrainData(torch.FloatTensor(X1),torch.FloatTensor(y))
train_l=round(int(cs.__len__())*0.8)

batch_size=25
val_l=int(cs.__len__())-train_l

train_set, val_set = torch.utils.data.random_split(cs, [train_l, val_l])

train_loader = DataLoader(train_set, shuffle=True, batch_size=batch_size, drop_last=True)


for each_x, each_y in train_loader:
    print(each_x.shape)
    print(each_y.shape)
    print(type(each_x))
    print(type(each_y))
    break
    
    
val_loader = DataLoader(val_set, shuffle=True, batch_size=batch_size, drop_last=True)
for each_x, each_y in val_loader:
    print(each_x.shape)
    print(each_y.shape)
    print(type(each_x))
    print(type(each_y))
    break

In [ ]:



# batch_size=25

# train_loader = DataLoader(dataset=train_data, shuffle=True, batch_size=batch_size, drop_last=True)


# val_loader = DataLoader(dataset=test_data, shuffle=True, batch_size=batch_size, drop_last=True)
# print(next(iter(train_loader))[0].shape[0])
# #print(train_loader.batch_size)
# for each_x, each_y in train_loader:
#     print(each_x.shape)
#     print(each_y.shape)
#     break
    
    

# for each_x,each_y in val_loader:
#     print(each_x.shape)
#     print(each_y.shape)
#     break

In [ ]:
# torch.cuda.is_available() checks and returns a Boolean True if a GPU is available, else it'll return False
is_cuda = torch.cuda.is_available()

# If we have a GPU available, we'll set our device to GPU. We'll use this device variable later in our code.
if is_cuda:
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

In [ ]:
class GRUNet(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, n_layers, drop_prob=0.2):
        super(GRUNet, self).__init__()
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        
        self.gru = nn.GRU(input_dim, hidden_dim, n_layers, batch_first=True, dropout=drop_prob)
        self.fc = nn.Linear(hidden_dim, output_dim)
        self.relu = nn.ReLU()
        
    def forward(self, x, h):
        #print(x,h)
        out, h = self.gru(x, h)
        out = self.fc(self.relu(out[:,-1]))
        return out, h
    
    def init_hidden(self, batch_size):
        weight = next(self.parameters()).data
        hidden = weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().to(device)
        return hidden
class LSTMNet(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, n_layers, drop_prob=0.2):
        super(LSTMNet, self).__init__()
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        
        self.lstm = nn.LSTM(input_dim, hidden_dim, n_layers, batch_first=True, dropout=drop_prob)
        self.fc = nn.Linear(hidden_dim, output_dim)
        self.relu = nn.ReLU()
        
    def forward(self, x, h):
        out, h = self.lstm(x, h)
        out = self.fc(self.relu(out[:,-1]))
        return out, h
    
    def init_hidden(self, batch_size):
        weight = next(self.parameters()).data
        hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().to(device),
                  weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().to(device))
        return hidden

In [ ]:

def train(train_loader, learn_rate, hidden_dim=256, EPOCHS=5, model_type="GRU"):
    print(next(iter(train_loader))[0].shape[2])
    # Setting common hyperparameters
    input_dim = next(iter(train_loader))[0].shape[2]
    output_dim = 3
    n_layers = 1
    # Instantiating the models
    if model_type == "GRU":
        model = GRUNet(input_dim, hidden_dim, output_dim, n_layers)
    else:
        model = LSTMNet(input_dim, hidden_dim, output_dim, n_layers)
    model.to(device)
    
    # Defining loss function and optimizer
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learn_rate)
    
    model.train()
    print("Starting Training of {} model".format(model_type))
    epoch_times = []
    # Start training loop
    for epoch in range(1,EPOCHS+1):
        start_time = time.time()
        h = model.init_hidden(batch_size)
        avg_loss = 0.
        counter = 0
        for x, label in train_loader:
            counter += 1
            if model_type == "GRU":
                h = h.data
            else:
                h = tuple([e.data for e in h])
            model.zero_grad()
            
            out, h = model(x.to(device), h)
            loss = criterion(out, label.to(device))
            loss.backward()
            optimizer.step()
            avg_loss += loss.item()
            if counter%200 == 0:
                print("Epoch {}......Step: {}/{}....... Average Loss for Epoch: {}".format(epoch, counter, len(train_loader), avg_loss/counter))
        current_time = time.time()
        print("Epoch {}/{} Done, Total Loss: {}".format(epoch, EPOCHS, avg_loss/len(train_loader)))
        print("Total Time Elapsed: {} seconds".format(str(current_time-start_time)))
        epoch_times.append(current_time-start_time)
    print("Total Training Time: {} seconds".format(str(sum(epoch_times))))
    return model


In [ ]:
def evaluate(model, val_loader, label_scalers):
    model.eval()
    outputs = []
    targets = []
    start_time = time.time()
    for each_x,each_y in val_loader:
        inp = (each_x.to(device))
        labs = (each_y.to(device))
        h = model.init_hidden(inp.shape[0])
        # print(inp.size)
        out, h = model(inp, h)
        outputs.extend(out.cpu().detach().numpy())#.reshape(1, -1)[0])
        targets.extend(each_y.cpu().numpy())#.reshape(1, -1)[0])
    print("Evaluation Time: {}".format(str(time.time()-start_time)))
    
    return outputs, targets

In [ ]:
lr = 0.001
gru_model = train(train_loader, lr, model_type="GRU")
#Lstm_model = train(train_loader, lr, model_type="LSTM")

In [ ]:
gru_outputs, targets = evaluate(gru_model, val_loader, label_scalers=y_te)



In [ ]:
print(targets[3])

In [ ]:
plt.plot( targets)

In [ ]:
plt.plot(gru_outputs)

In [ ]:

data=pd.dataframe(gru_outputs)
data.head()

In [ ]:
output=np.array(gru_outputs)
data = pd.DataFrame(output, columns = ['base','rotation','translation'])
data.head()


In [ ]:
out=np.array(targets)
dat = pd.DataFrame(out, columns = ['base1','rotation1','translation1'])
dat.head()



In [ ]:
from cmath import tanh
r=[]

res = output.max(axis=1)
print(type(res))
for i in res:
    print(i)
#     r.append (tanh(res[i]))    
plt.plot(res)

In [ ]:
data['base1'] = dat.base1
data['rotation1'] = dat.rotation1
data['translation1'] = dat.translation1
data.head()

In [ ]:
data.to_csv("results_1.csv")